In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import re
from io import StringIO
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime


FILENAME = './data/train.csv'
with open(FILENAME) as file:
  lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
  df = pd.read_csv(StringIO(''.join(lines)), escapechar="/")


# Example function to load and preprocess images
def load_and_preprocess_image(file_path, target_size=(220, 220)):
    img = tf.keras.preprocessing.image.load_img(file_path, target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    return img_array

# Apply preprocessing to all images
X = np.array([load_and_preprocess_image("./data/images/{}".format(fp)) for fp in df['ImageID']])
y = np.array([eval(label_str) for label_str in df['Labels']])  # Assuming 'labels' is a string representation of a list

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Load EfficientNetB0 with pre-trained ImageNet weights
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(320, 320, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model's layers initially
base_model.trainable = False

# Create the classification model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(19, activation='sigmoid')  # Adjust to your number of classes
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks setup
checkpoint = ModelCheckpoint(
    'best_model.h5',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1  # Verbose mode for ModelCheckpoint
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
    verbose=1  # Verbose mode for EarlyStopping
)

# TensorBoard logs
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training the model with verbosity and callbacks
print("Starting initial training with frozen layers...")
model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint, early_stopping, tensorboard_callback],
    verbose=2  # Verbose mode for training (2 = detailed output)
)

# Optionally, set the base model's layers to trainable and continue training with a lower learning rate for further fine-tuning
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Starting fine-tuning with unfrozen base model layers...")
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint, early_stopping, tensorboard_callback],
    verbose=2
)

print("Training completed.")


MemoryError: Unable to allocate 16.2 GiB for an array with shape (29996, 220, 220, 3) and data type float32

# Hugging Face tutorial docs

In [13]:
import torch
from datasets import load_dataset
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
import matplotlib.pyplot as plt
def display_image(file_name):
    # Display the image
    plt.imshow(image)
    plt.axis('off')  # Turn off axis
    plt.show()

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

preprocessor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b0")
model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b0")




c:\Users\eoinp\anaconda3\envs\deepl\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for huggingface/cats-image contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/huggingface/cats-image
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
c:\Users\eoinp\anaconda3\envs\deepl\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
from PIL import Image
url = "./data/images/10.jpg"
image = Image.open(url)
inputs = preprocessor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label]),

pizza, pizza pie


(None,)

In [25]:
display_image(image)
print(logits)

tensor([[-3.3413e-01,  6.8951e-01, -8.5977e-02,  1.7073e-01, -4.2301e-01,
          2.5094e+00,  1.4965e-01, -2.7562e-01,  6.4496e-01, -7.3586e-02,
          8.6784e-01, -6.4929e-01, -1.8429e-03, -7.2723e-01, -2.8456e-01,
         -2.7268e-01, -7.9624e-01, -1.3005e+00, -5.6836e-01, -2.8079e-01,
         -3.7753e-01,  2.1212e-01,  7.6995e-02, -3.1726e-01,  5.7034e-01,
         -1.4163e+00, -4.7424e-01, -7.4053e-01, -1.3076e+00, -6.2832e-01,
         -1.5136e-01, -2.1876e+00, -4.9105e-01,  2.4147e+00,  1.7611e+00,
         -1.1911e-01, -3.8646e-01,  2.2227e-01, -5.4684e-01,  6.9896e-01,
         -1.0222e+00, -2.4194e-01, -4.2299e-01,  4.2813e-01, -8.7290e-01,
          1.0488e+00, -9.9214e-01,  8.1178e-01, -7.7554e-01, -7.4287e-01,
         -7.4637e-01, -6.5330e-01, -7.2701e-01, -5.6594e-01, -6.5094e-01,
         -1.3358e+00, -1.0896e+00, -2.2738e-01, -5.4973e-01, -7.1332e-01,
          5.2822e-02, -1.0013e+00, -1.0310e+00,  1.1579e+00, -7.5359e-01,
         -1.5547e+00, -3.7829e-02, -2.

In [16]:
print("Existing ImageNet labels:")
for idx, label in model.config.id2label.items():
    print(f"{idx}: {label}")

Existing ImageNet labels:
0: tench, Tinca tinca
1: goldfish, Carassius auratus
2: great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias
3: tiger shark, Galeocerdo cuvieri
4: hammerhead, hammerhead shark
5: electric ray, crampfish, numbfish, torpedo
6: stingray
7: cock
8: hen
9: ostrich, Struthio camelus
10: brambling, Fringilla montifringilla
11: goldfinch, Carduelis carduelis
12: house finch, linnet, Carpodacus mexicanus
13: junco, snowbird
14: indigo bunting, indigo finch, indigo bird, Passerina cyanea
15: robin, American robin, Turdus migratorius
16: bulbul
17: jay
18: magpie
19: chickadee
20: water ouzel, dipper
21: kite
22: bald eagle, American eagle, Haliaeetus leucocephalus
23: vulture
24: great grey owl, great gray owl, Strix nebulosa
25: European fire salamander, Salamandra salamandra
26: common newt, Triturus vulgaris
27: eft
28: spotted salamander, Ambystoma maculatum
29: axolotl, mud puppy, Ambystoma mexicanum
30: bullfrog, Rana catesbeiana
31: